In [1]:
import pandas as pd
import os

In [2]:
data1 = pd.read_csv(os.path.join('datasets', 'reservoir', 'reservoir_train400_scenario1.csv'))
data1.head()

,PORO MATRIX,PERMX MATRIX (mD),PORO CRACK,PERMX CRACK (mD),OIL RATE 0 (SM3/DAY),OIL RATE 1 (SM3/DAY),OIL RATE 2 (SM3/DAY),OIL RATE 3 (SM3/DAY),OIL RATE 4 (SM3/DAY),OIL RATE 5 (SM3/DAY),...,OIL RATE 51 (SM3/DAY),OIL RATE 52 (SM3/DAY),OIL RATE 53 (SM3/DAY),OIL RATE 54 (SM3/DAY),OIL RATE 55 (SM3/DAY),OIL RATE 56 (SM3/DAY),OIL RATE 57 (SM3/DAY),OIL RATE 58 (SM3/DAY),OIL RATE 59 (SM3/DAY),OIL RATE 60 (SM3/DAY)
0,0.21100,2.17400,0.014675,1400.375,0,67.43562,131.4994,183.2932,224.5630,249.7905,...,85.50780,82.61002,79.99155,77.52847,74.87337,72.41143,69.99726,67.62505,65.28770,63.00892
1,0.19225,8.27100,0.010400,1483.250,0,64.59608,123.5471,171.1804,209.6795,238.2284,...,51.93359,48.90244,46.20414,43.79527,41.34522,38.86421,36.20404,33.51224,31.35427,29.51257
2,0.21975,6.38275,0.012200,1410.125,0,59.89659,116.4695,163.4962,202.2798,231.0852,...,84.37846,79.25503,73.43829,69.06870,65.60688,62.29403,58.89056,55.69555,52.56306,49.41299
3,0.17725,4.99500,0.037062,435.125,0,64.23164,128.6341,179.6088,221.4585,256.1675,...,103.99100,99.48777,95.12907,91.19880,86.97459,81.89688,75.66498,70.85409,66.96544,63.26744
4,0.20625,9.22650,0.034362,1088.375,0,140.04380,227.9327,286.8897,328.3230,357.4857,...,120.93770,115.23790,108.47000,100.44030,94.15714,88.79877,83.78366,79.06993,74.59715,70.24670


In [3]:
data2 = pd.read_csv(os.path.join('datasets', 'reservoir', 'reservoir_train800_scenario1.csv'))
data2.head()

,PORO MATRIX,PERMX MATRIX (mD),PORO CRACK,PERMX CRACK (mD),OIL RATE 0 (SM3/DAY),OIL RATE 1 (SM3/DAY),OIL RATE 2 (SM3/DAY),OIL RATE 3 (SM3/DAY),OIL RATE 4 (SM3/DAY),OIL RATE 5 (SM3/DAY),...,OIL RATE 51 (SM3/DAY),OIL RATE 52 (SM3/DAY),OIL RATE 53 (SM3/DAY),OIL RATE 54 (SM3/DAY),OIL RATE 55 (SM3/DAY),OIL RATE 56 (SM3/DAY),OIL RATE 57 (SM3/DAY),OIL RATE 58 (SM3/DAY),OIL RATE 59 (SM3/DAY),OIL RATE 60 (SM3/DAY)
0,0.177500,8.350625,0.038413,444.8750,0,74.95135,142.45130,194.79360,237.0151,271.2165,...,94.13045,89.59142,85.31583,81.22491,77.40237,73.93965,70.58395,67.25111,63.99178,60.88605
1,0.178375,2.321875,0.013213,596.0000,0,23.01975,56.40776,89.68631,120.0415,147.0736,...,72.21385,67.72377,64.99429,62.12570,59.13687,56.45981,53.95433,51.39895,48.96900,46.74735
2,0.170875,4.938125,0.027669,335.1875,0,35.48324,79.75836,119.16950,153.5473,183.8696,...,76.38937,72.48967,68.74857,65.25943,62.06694,58.87563,55.82406,53.00330,50.43834,48.06820
3,0.162750,5.950500,0.041787,286.4375,0,52.27187,108.97820,155.87210,195.1530,228.5208,...,91.59255,87.15457,82.99065,79.08322,75.34554,71.77395,68.51920,65.43627,62.23518,59.28650
4,0.154000,2.765500,0.021200,1115.1875,0,107.60920,191.04990,252.18310,297.5018,326.4799,...,58.09989,55.13623,52.43000,49.92253,47.59525,45.41184,43.23558,41.07516,38.97419,36.95325


In [4]:
data3 = pd.read_csv(os.path.join('datasets', 'reservoir', 'reservoir_train800_scenario2.csv'))
data3.head()

,PORO MATRIX,PERMX MATRIX (mD),PORO CRACK,PERMX CRACK (mD),OIL RATE 0 (SM3/DAY),OIL RATE 1 (SM3/DAY),OIL RATE 2 (SM3/DAY),OIL RATE 3 (SM3/DAY),OIL RATE 4 (SM3/DAY),OIL RATE 5 (SM3/DAY),...,OIL RATE 51 (SM3/DAY),OIL RATE 52 (SM3/DAY),OIL RATE 53 (SM3/DAY),OIL RATE 54 (SM3/DAY),OIL RATE 55 (SM3/DAY),OIL RATE 56 (SM3/DAY),OIL RATE 57 (SM3/DAY),OIL RATE 58 (SM3/DAY),OIL RATE 59 (SM3/DAY),OIL RATE 60 (SM3/DAY)
0,0.177500,8.350625,0.038413,444.8750,0,452.2205,281.9055,181.4378,117.53240,76.46229,...,137.57210,130.15880,123.12450,116.42750,110.18060,104.29040,98.79874,93.62064,88.62532,83.99811
1,0.178375,2.321875,0.013213,596.0000,0,237.2867,191.7555,154.3891,125.15420,101.81400,...,123.60040,118.30710,113.19480,108.21250,103.49630,99.05050,94.80075,90.85400,86.65803,82.37943
2,0.170875,4.938125,0.027669,335.1875,0,301.1440,227.8870,172.7210,132.00200,101.21170,...,138.79240,130.94050,123.44120,116.36760,109.73140,103.49240,97.64111,92.03975,86.80898,81.93594
3,0.162750,5.950500,0.041787,286.4375,0,360.9188,249.5488,175.5890,124.06210,87.96030,...,147.82040,139.92880,132.47420,125.40490,118.69870,112.33430,106.39500,100.81110,95.55692,90.56459
4,0.154000,2.765500,0.021200,1115.1875,0,420.1661,219.5578,123.0940,69.81461,40.03331,...,74.58723,69.90481,65.63278,63.09455,60.21962,57.25322,54.44020,51.68710,49.04267,46.67566
